In [5]:
"""
================================================================================
PASO 10: ANÁLISIS AVANZADO Y VALIDACIÓN FINAL - VERSION COMPLETA
================================================================================

Proyecto: Predicción de Síndrome de Ovario Poliquístico (SOP)
Institución: Clúster de Ingeniería Biomédica del Estado de Jalisco
Fecha: Noviembre 2025

INCLUYE:
- Métricas clínicas extendidas (Sens/Spec/PPV/NPV/LR+/LR-)
- SHAP explicabilidad (XGBoost + Random Forest)
- Comparación de modelos (LR, RF, XGBoost, KNN)
- Validación cruzada repetida (5×3)
- Reporte final JSON

NOTA: Ensemble eliminado (conflictos sklearn/xgboost, no mejora resultados)
================================================================================
"""

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

from sklearn.model_selection import (
    train_test_split, StratifiedKFold, RepeatedStratifiedKFold,
    cross_validate, GridSearchCV
)
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)

from imblearn.over_sampling import SMOTE

try:
    import shap
    SHAP_OK = True
except ImportError:
    SHAP_OK = False

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("="*80)
print("PASO 10: ANÁLISIS AVANZADO Y VALIDACIÓN FINAL")
print("="*80)
print()

if SHAP_OK:
    print("✓ SHAP disponible")
else:
    print("⚠️ SHAP no disponible (pip install shap)")
print()

# =============================================================================
# SECCIÓN 1: CARGA Y PREPARACIÓN DE DATOS
# =============================================================================

print("="*80)
print("SECCIÓN 1: CARGA Y PREPARACIÓN")
print("="*80)
print()

PATH_DATASET_A = '../documentos_generados/PCOS_data_transformado.csv'
PATH_DATASET_B = 'PCOS_data_FINAL_sin_multicolinealidad.csv'
TARGET = 'SOP (S/N)'

print("Cargando datasets...")
df_trees = pd.read_csv(PATH_DATASET_A)
df_logit = pd.read_csv(PATH_DATASET_B)

# Conversión a float64
print("Convirtiendo a float64...")
for col in df_trees.columns:
    if col != TARGET:
        df_trees[col] = pd.to_numeric(df_trees[col], errors='coerce').fillna(0).astype(np.float64)

for col in df_logit.columns:
    if col != TARGET:
        df_logit[col] = pd.to_numeric(df_logit[col], errors='coerce').fillna(0).astype(np.float64)

print(f"✓ Dataset A: {df_trees.shape}")
print(f"✓ Dataset B: {df_logit.shape}")
print()

# Guardar nombres de columnas
feature_names_trees = df_trees.drop(TARGET, axis=1).columns.tolist()
feature_names_logit = df_logit.drop(TARGET, axis=1).columns.tolist()

# Convertir a arrays
X_trees = df_trees.drop(TARGET, axis=1).values.astype(np.float64)
y_trees = df_trees[TARGET].values.astype(np.int32)

X_logit = df_logit.drop(TARGET, axis=1).values.astype(np.float64)
y_logit = df_logit[TARGET].values.astype(np.int32)

# Split
X_train_trees, X_test_trees, y_train_trees, y_test_trees = train_test_split(
    X_trees, y_trees, test_size=0.20, random_state=RANDOM_STATE, stratify=y_trees
)

X_train_logit, X_test_logit, y_train_logit, y_test_logit = train_test_split(
    X_logit, y_logit, test_size=0.20, random_state=RANDOM_STATE, stratify=y_logit
)

# SMOTE
print("Aplicando SMOTE...")
smote = SMOTE(random_state=RANDOM_STATE)

X_train_trees_bal, y_train_trees_bal = smote.fit_resample(X_train_trees, y_train_trees)
X_train_trees_bal = X_train_trees_bal.astype(np.float64)

X_train_logit_bal, y_train_logit_bal = smote.fit_resample(X_train_logit, y_train_logit)
X_train_logit_bal = X_train_logit_bal.astype(np.float64)

# Escalamiento
scaler_logit = StandardScaler()
X_train_logit_scaled = scaler_logit.fit_transform(X_train_logit_bal).astype(np.float64)
X_test_logit_scaled = scaler_logit.transform(X_test_logit).astype(np.float64)

scaler_knn = StandardScaler()
X_train_trees_scaled = scaler_knn.fit_transform(X_train_trees_bal).astype(np.float64)
X_test_trees_scaled = scaler_knn.transform(X_test_trees).astype(np.float64)

print(f"✓ Train={len(X_train_trees_bal)}, Test={len(X_test_trees)}")
print()

# =============================================================================
# SECCIÓN 2: ENTRENAMIENTO DE MODELOS
# =============================================================================

print("="*80)
print("SECCIÓN 2: ENTRENAMIENTO DE MODELOS")
print("="*80)
print()

# 1. Logistic Regression
print("1. Logistic Regression...")
lr_model = LogisticRegression(C=0.1, max_iter=1000, random_state=RANDOM_STATE)
lr_model.fit(X_train_logit_scaled, y_train_logit_bal)
y_pred_lr = lr_model.predict(X_test_logit_scaled)
y_pred_proba_lr = lr_model.predict_proba(X_test_logit_scaled)[:, 1]

# 2. Random Forest con RFE (30 features)
print("2. Random Forest con RFE (30 features)...")
rf_base = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rfe = RFE(rf_base, n_features_to_select=30, step=1)
rfe.fit(X_train_trees_bal, y_train_trees_bal)

X_train_rf = rfe.transform(X_train_trees_bal).astype(np.float64)
X_test_rf = rfe.transform(X_test_trees).astype(np.float64)

selected_features = [feature_names_trees[i] for i, sel in enumerate(rfe.support_) if sel]

rf_model = RandomForestClassifier(
    n_estimators=200, max_depth=10, min_samples_split=5, 
    min_samples_leaf=1, random_state=RANDOM_STATE
)
rf_model.fit(X_train_rf, y_train_trees_bal)
y_pred_rf = rf_model.predict(X_test_rf)
y_pred_proba_rf = rf_model.predict_proba(X_test_rf)[:, 1]

# 3. XGBoost (con las mismas 30 features que RF)
print("3. XGBoost (30 features)...")
xgb_model = xgb.XGBClassifier(
    learning_rate=0.1, n_estimators=100, max_depth=5,
    subsample=0.8, random_state=RANDOM_STATE, 
    eval_metric='logloss', base_score=0.5, use_label_encoder=False
)
xgb_model.fit(X_train_rf, y_train_trees_bal)
y_pred_xgb = xgb_model.predict(X_test_rf)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_rf)[:, 1]

# 3b. XGBoost COMPLETO (41 features) - solo para SHAP
print("3b. XGBoost completo (41 features, solo para SHAP)...")
xgb_model_full = xgb.XGBClassifier(
    learning_rate=0.1, n_estimators=100, max_depth=5,
    subsample=0.8, random_state=RANDOM_STATE, 
    eval_metric='logloss', base_score=0.5, use_label_encoder=False
)
xgb_model_full.fit(X_train_trees_bal.astype(np.float64), y_train_trees_bal)
y_pred_xgb_full = xgb_model_full.predict(X_test_trees)
y_pred_proba_xgb_full = xgb_model_full.predict_proba(X_test_trees)[:, 1]

# 4. KNN
print("4. K-Nearest Neighbors...")
knn_model = KNeighborsClassifier(n_neighbors=7, weights='distance', metric='manhattan')
knn_model.fit(X_train_trees_scaled, y_train_trees_bal)
y_pred_knn = knn_model.predict(X_test_trees_scaled)
y_pred_proba_knn = knn_model.predict_proba(X_test_trees_scaled)[:, 1]

print("✓ Modelos entrenados")
print()
print("NOTA: Ensemble eliminado (causa conflictos sklearn/xgboost y no mejora resultados)")
print()

# =============================================================================
# SECCIÓN 3: MÉTRICAS CLÍNICAS EXTENDIDAS
# =============================================================================

print("="*80)
print("SECCIÓN 3: MÉTRICAS CLÍNICAS EXTENDIDAS")
print("="*80)
print()

def calcular_metricas_clinicas(y_true, y_pred, y_pred_proba, model_name):
    """Calcula métricas clínicas completas"""
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    sensibilidad = tp / (tp + fn) if (tp + fn) > 0 else 0
    especificidad = tn / (tn + fp) if (tn + fp) > 0 else 0
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    lr_pos = sensibilidad / (1 - especificidad) if especificidad < 1 else np.inf
    lr_neg = (1 - sensibilidad) / especificidad if especificidad > 0 else np.inf
    
    return {
        'Model': model_name,
        'TN': int(tn), 'FP': int(fp), 'FN': int(fn), 'TP': int(tp),
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision (PPV)': ppv,
        'Recall (Sensibilidad)': sensibilidad,
        'Especificidad': especificidad,
        'F1-Score': f1_score(y_true, y_pred),
        'NPV': npv,
        'LR+': lr_pos,
        'LR-': lr_neg,
        'ROC-AUC': roc_auc_score(y_true, y_pred_proba)
    }

metricas_clinicas = []
metricas_clinicas.append(calcular_metricas_clinicas(y_test_logit, y_pred_lr, y_pred_proba_lr, 'Logistic Regression'))
metricas_clinicas.append(calcular_metricas_clinicas(y_test_trees, y_pred_rf, y_pred_proba_rf, 'Random Forest (30f)'))
metricas_clinicas.append(calcular_metricas_clinicas(y_test_trees, y_pred_xgb_full, y_pred_proba_xgb_full, 'XGBoost (41f)'))
metricas_clinicas.append(calcular_metricas_clinicas(y_test_trees, y_pred_xgb, y_pred_proba_xgb, 'XGBoost (30f)'))
metricas_clinicas.append(calcular_metricas_clinicas(y_test_trees, y_pred_knn, y_pred_proba_knn, 'KNN'))

df_clinicas = pd.DataFrame(metricas_clinicas)
print(df_clinicas.to_string(index=False))
print()

df_clinicas.to_csv('metricas_clinicas_extendidas.csv', index=False)
print("✓ Guardado: metricas_clinicas_extendidas.csv")
print()

print("NOTA: XGBoost evaluado con 41 features (completo) y 30 features (RFE)")
print()

print("INTERPRETACIÓN CLÍNICA:")
print()
print("Sensibilidad (Recall): % de casos SOP detectados correctamente")
print("  → Alto = Menos falsos negativos (no se pierden casos)")
print()
print("Especificidad: % de No-SOP identificados correctamente")
print("  → Alto = Menos falsos positivos (menos alarmas falsas)")
print()
print("PPV (Precision): Si predice SOP, ¿probabilidad real?")
print("  → Alto = Predicción SOP confiable")
print()
print("NPV: Si predice No-SOP, ¿probabilidad real?")
print("  → Alto = Predicción negativa confiable")
print()
print("LR+ (Likelihood Ratio +): >10 excelente, >5 muy bueno")
print("LR- (Likelihood Ratio -): <0.1 excelente, <0.2 muy bueno")
print()

# =============================================================================
# SECCIÓN 4: SHAP EXPLICABILIDAD
# =============================================================================

print("="*80)
print("SECCIÓN 4: SHAP EXPLICABILIDAD")
print("="*80)
print()

if not SHAP_OK:
    print("⚠️ SHAP no disponible")
    print("   Instalar: pip install shap --break-system-packages")
    print()
else:
    try:
        # XGBoost SHAP (con 41 features completas)
        print("Calculando SHAP para XGBoost (41 features)...")
        print("(Esto puede tardar 2-3 minutos)")
        print()
        
        X_test_trees_clean = X_test_trees.astype(np.float64)
        
        explainer_xgb = shap.TreeExplainer(xgb_model_full)
        shap_values_xgb = explainer_xgb.shap_values(X_test_trees_clean)
        
        # Summary plot (barras)
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_xgb, X_test_trees_clean,
                         feature_names=feature_names_trees,
                         plot_type="bar", show=False, max_display=15)
        plt.title('XGBoost - Top 15 Variables Importantes (SHAP)', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('shap_xgboost_summary.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ shap_xgboost_summary.png")
        
        # Beeswarm plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_xgb, X_test_trees_clean,
                         feature_names=feature_names_trees,
                         show=False, max_display=15)
        plt.title('XGBoost - Distribución de Impacto (SHAP)', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('shap_xgboost_beeswarm.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ shap_xgboost_beeswarm.png")
        
        # Tabla importancia
        shap_importance_xgb = pd.DataFrame({
            'Feature': feature_names_trees,
            'SHAP_Mean_Abs': np.abs(shap_values_xgb).mean(axis=0)
        }).sort_values('SHAP_Mean_Abs', ascending=False)
        
        shap_importance_xgb.to_csv('shap_importance_xgboost.csv', index=False)
        print("✓ shap_importance_xgboost.csv")
        print()
        
        print("TOP 10 VARIABLES (XGBoost SHAP):")
        print(shap_importance_xgb.head(10).to_string(index=False))
        print()
        
        # Random Forest SHAP
        print("Calculando SHAP para Random Forest...")
        print("(Esto puede tardar 3-5 minutos)")
        print()
        
        explainer_rf = shap.TreeExplainer(rf_model)
        shap_values_rf = explainer_rf.shap_values(X_test_rf)
        
        if isinstance(shap_values_rf, list):
            shap_values_rf = shap_values_rf[1]
        
        # Summary plot RF
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_rf, X_test_rf,
                         feature_names=selected_features,
                         plot_type="bar", show=False, max_display=15)
        plt.title('Random Forest (30f) - Top 15 Variables (SHAP)', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('shap_rf_summary.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ shap_rf_summary.png")
        
        # Beeswarm RF
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_rf, X_test_rf,
                         feature_names=selected_features,
                         show=False, max_display=15)
        plt.title('Random Forest (30f) - Distribución de Impacto (SHAP)', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('shap_rf_beeswarm.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ shap_rf_beeswarm.png")
        
        # Tabla importancia RF
        shap_importance_rf = pd.DataFrame({
            'Feature': selected_features,
            'SHAP_Mean_Abs': np.abs(shap_values_rf).mean(axis=0)
        }).sort_values('SHAP_Mean_Abs', ascending=False)
        
        shap_importance_rf.to_csv('shap_importance_rf.csv', index=False)
        print("✓ shap_importance_rf.csv")
        print()
        
        print("TOP 10 VARIABLES (Random Forest SHAP):")
        print(shap_importance_rf.head(10).to_string(index=False))
        print()
        
        print("✅ SHAP completado exitosamente")
        print()
        
    except Exception as e:
        print(f"❌ Error en SHAP: {e}")
        print()

# =============================================================================
# SECCIÓN 5: COMPARACIÓN DE MODELOS
# =============================================================================

print("="*80)
print("SECCIÓN 5: COMPARACIÓN DE MODELOS")
print("="*80)
print()

comparison_data = []
for name, y_pred, y_proba, y_true in [
    ('Logistic Regression', y_pred_lr, y_pred_proba_lr, y_test_logit),
    ('Random Forest (30f)', y_pred_rf, y_pred_proba_rf, y_test_trees),
    ('XGBoost (41f)', y_pred_xgb_full, y_pred_proba_xgb_full, y_test_trees),
    ('XGBoost (30f)', y_pred_xgb, y_pred_proba_xgb, y_test_trees),
    ('KNN', y_pred_knn, y_pred_proba_knn, y_test_trees)
]:
    comparison_data.append({
        'Model': name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1-Score': f1_score(y_true, y_pred),
        'ROC-AUC': roc_auc_score(y_true, y_proba)
    })

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))
print()

df_comparison.to_csv('modelos_comparison.csv', index=False)
print("✓ Guardado: modelos_comparison.csv")
print()

# Identificar mejor modelo en test
best_model_test = df_comparison.loc[df_comparison['F1-Score'].idxmax(), 'Model']
best_f1_test = df_comparison['F1-Score'].max()

print(f"🏆 MEJOR MODELO EN TEST SET: {best_model_test} (F1={best_f1_test:.4f})")
print()

# =============================================================================
# SECCIÓN 6: VALIDACIÓN CRUZADA REPETIDA
# =============================================================================

print("="*80)
print("SECCIÓN 6: VALIDACIÓN CRUZADA REPETIDA (5×3)")
print("="*80)
print()

print("Configuración: 5-Fold × 3 repeticiones = 15 evaluaciones")
print("(Esto puede tardar 5-10 minutos)")
print()

repeated_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=RANDOM_STATE)

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

modelos_validacion = {
    'Logistic Regression': (lr_model, X_train_logit_scaled, y_train_logit_bal),
    'Random Forest (30f)': (rf_model, X_train_rf, y_train_trees_bal),
    'XGBoost (30f)': (xgb_model, X_train_rf, y_train_trees_bal),
    'KNN': (knn_model, X_train_trees_scaled, y_train_trees_bal)
}

resultados_validacion = []

for model_name, (model, X_train, y_train) in modelos_validacion.items():
    print(f"Validando {model_name}...")
    
    cv_results = cross_validate(
        model, X_train, y_train,
        cv=repeated_cv,
        scoring=scoring,
        n_jobs=-1
    )
    
    resultados_validacion.append({
        'Model': model_name,
        'Accuracy_mean': cv_results['test_accuracy'].mean(),
        'Accuracy_std': cv_results['test_accuracy'].std(),
        'Precision_mean': cv_results['test_precision'].mean(),
        'Precision_std': cv_results['test_precision'].std(),
        'Recall_mean': cv_results['test_recall'].mean(),
        'Recall_std': cv_results['test_recall'].std(),
        'F1_mean': cv_results['test_f1'].mean(),
        'F1_std': cv_results['test_f1'].std(),
        'ROC_AUC_mean': cv_results['test_roc_auc'].mean(),
        'ROC_AUC_std': cv_results['test_roc_auc'].std()
    })
    
    print(f"  ✓ F1 = {cv_results['test_f1'].mean():.4f} ± {cv_results['test_f1'].std():.4f}")

print()

df_validacion = pd.DataFrame(resultados_validacion)
print("RESULTADOS VALIDACIÓN CRUZADA REPETIDA:")
print()
print(df_validacion.to_string(index=False))
print()

df_validacion.to_csv('validacion_repetida_results.csv', index=False)
print("✓ Guardado: validacion_repetida_results.csv")
print()

# Visualización intervalos de confianza
fig, ax = plt.subplots(figsize=(12, 6))

models = df_validacion['Model'].values
f1_means = df_validacion['F1_mean'].values
f1_stds = df_validacion['F1_std'].values

y_pos = np.arange(len(models))

ax.barh(y_pos, f1_means, xerr=f1_stds, align='center', alpha=0.7, 
        capsize=5, color='steelblue', ecolor='darkblue')
ax.set_yticks(y_pos)
ax.set_yticklabels(models)
ax.invert_yaxis()
ax.set_xlabel('F1-Score (media ± std)', fontsize=12, fontweight='bold')
ax.set_title('Validación Cruzada Repetida - F1-Score con Intervalos de Confianza', 
             fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('validacion_repetida_intervals.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Guardado: validacion_repetida_intervals.png")
print()

# =============================================================================
# SECCIÓN 7: REPORTE FINAL Y RECOMENDACIONES
# =============================================================================

print("="*80)
print("REPORTE FINAL Y RECOMENDACIONES")
print("="*80)
print()

# Identificar mejor modelo
best_model_idx = df_validacion['F1_mean'].idxmax()
best_model_name = df_validacion.loc[best_model_idx, 'Model']
best_f1_mean = df_validacion.loc[best_model_idx, 'F1_mean']
best_f1_std = df_validacion.loc[best_model_idx, 'F1_std']

print(f"🏆 MODELO RECOMENDADO:")
print(f"   {best_model_name}")
print(f"   F1-Score: {best_f1_mean:.4f} ± {best_f1_std:.4f}")
print()

# Obtener métricas del mejor modelo
best_metrics = [m for m in metricas_clinicas if m['Model'] == best_model_name][0]

print("📊 MÉTRICAS CLÍNICAS (Test Set):")
print(f"   Sensibilidad: {best_metrics['Recall (Sensibilidad)']:.1%}")
print(f"   Especificidad: {best_metrics['Especificidad']:.1%}")
print(f"   PPV: {best_metrics['Precision (PPV)']:.1%}")
print(f"   NPV: {best_metrics['NPV']:.1%}")
print(f"   LR+: {best_metrics['LR+']:.2f}")
print(f"   LR-: {best_metrics['LR-']:.3f}")
print()

print("📋 RECOMENDACIONES PARA TESIS/PAPER:")
print()
print("1. MODELO FINAL:")
print(f"   → Usar {best_model_name}")
print(f"   → Reportar F1 = {best_f1_mean:.4f} ± {best_f1_std:.4f}")
print(f"   → Mencionar validación cruzada repetida 5×3")
print()

print("2. MÉTRICAS CLÍNICAS A REPORTAR:")
print(f"   → Sensibilidad: {best_metrics['Recall (Sensibilidad)']:.1%} (detecta casos SOP)")
print(f"   → Especificidad: {best_metrics['Especificidad']:.1%} (identifica No-SOP)")
print(f"   → PPV: {best_metrics['Precision (PPV)']:.1%} (confianza predicción positiva)")
print(f"   → NPV: {best_metrics['NPV']:.1%} (confianza predicción negativa)")
print()

print("3. EXPLICABILIDAD:")
if SHAP_OK:
    print("   → Usar gráficos SHAP beeswarm para interpretación")
    print("   → Discutir top 5-10 variables más importantes")
    print("   → Validar coherencia clínica con criterios Rotterdam")
else:
    print("   → Instalar SHAP para generar gráficos de explicabilidad")
print()

print("4. VALIDACIÓN:")
print("   → Mencionar robustez (validación repetida 15 evaluaciones)")
print("   → Reportar intervalos de confianza")
print("   → Destacar reducción dimensional (42 → 30 features)")
print()

# Guardar reporte JSON
reporte_final = {
    'fecha_analisis': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'modelo_recomendado': {
        'nombre': best_model_name,
        'f1_score': {'mean': float(best_f1_mean), 'std': float(best_f1_std)},
        'validacion': 'Repeated Stratified K-Fold (5×3)'
    },
    'metricas_clinicas_test': {
        'sensibilidad': float(best_metrics['Recall (Sensibilidad)']),
        'especificidad': float(best_metrics['Especificidad']),
        'ppv': float(best_metrics['Precision (PPV)']),
        'npv': float(best_metrics['NPV']),
        'lr_plus': float(best_metrics['LR+']),
        'lr_minus': float(best_metrics['LR-']),
        'accuracy': float(best_metrics['Accuracy']),
        'roc_auc': float(best_metrics['ROC-AUC'])
    },
    'confusion_matrix_test': {
        'TN': int(best_metrics['TN']),
        'FP': int(best_metrics['FP']),
        'FN': int(best_metrics['FN']),
        'TP': int(best_metrics['TP'])
    },
    'features': {
        'originales': len(feature_names_trees),
        'seleccionadas_rf': 30 if 'RF' in best_model_name or 'Ensemble' in best_model_name else len(feature_names_trees),
        'metodo_seleccion': 'RFE' if 'RF' in best_model_name else 'None'
    },
    'comparacion_modelos': df_comparison.to_dict('records'),
    'validacion_cruzada': df_validacion.to_dict('records')
}

with open('reporte_final_paso10.json', 'w', encoding='utf-8') as f:
    json.dump(reporte_final, f, indent=2, ensure_ascii=False)

print("✓ Reporte guardado: reporte_final_paso10.json")
print()

# =============================================================================
# ARCHIVOS GENERADOS
# =============================================================================

print("="*80)
print("ARCHIVOS GENERADOS")
print("="*80)
print()

print("📁 MÉTRICAS Y RESULTADOS:")
print("  1. metricas_clinicas_extendidas.csv")
print("  2. modelos_comparison.csv")
print("  3. validacion_repetida_results.csv")
if SHAP_OK:
    print("  4. shap_importance_xgboost.csv")
    print("  5. shap_importance_rf.csv")
print()

print("📊 VISUALIZACIONES:")
if SHAP_OK:
    print("  6. shap_xgboost_summary.png")
    print("  7. shap_xgboost_beeswarm.png")
    print("  8. shap_rf_summary.png")
    print("  9. shap_rf_beeswarm.png")
print("  10. validacion_repetida_intervals.png")
print()

print("📄 REPORTES:")
print("  11. reporte_final_paso10.json")
print()

# =============================================================================
# FINALIZACIÓN
# =============================================================================

print("="*80)
print("✅ PASO 10 COMPLETADO EXITOSAMENTE")
print("="*80)
print()

print("🎉 ANÁLISIS COMPLETO Y VALIDADO")
print()
print("Has completado:")
print("  ✓ Métricas clínicas extendidas (Sens/Spec/PPV/NPV/LR)")
print("  ✓ Explicabilidad SHAP (XGBoost + Random Forest)")
print("  ✓ Comparación completa de modelos")
print("  ✓ Validación cruzada repetida (robustez demostrada)")
print("  ✓ Reporte final completo")
print()
print("Tu proyecto está listo para:")
print("  → Defensa de tesis ✅")
print("  → Publicación científica ✅")
print("  → Presentación a comité biomédico ✅")
print()
print("="*80)

PASO 10: ANÁLISIS AVANZADO Y VALIDACIÓN FINAL

✓ SHAP disponible

SECCIÓN 1: CARGA Y PREPARACIÓN

Cargando datasets...
Convirtiendo a float64...
✓ Dataset A: (538, 42)
✓ Dataset B: (538, 19)

Aplicando SMOTE...
✓ Train=578, Test=108

SECCIÓN 2: ENTRENAMIENTO DE MODELOS

1. Logistic Regression...
2. Random Forest con RFE (30 features)...
3. XGBoost (30 features)...
3b. XGBoost completo (41 features, solo para SHAP)...
4. K-Nearest Neighbors...
✓ Modelos entrenados

NOTA: Ensemble eliminado (causa conflictos sklearn/xgboost y no mejora resultados)

SECCIÓN 3: MÉTRICAS CLÍNICAS EXTENDIDAS

              Model  TN  FP  FN  TP  Accuracy  Precision (PPV)  Recall (Sensibilidad)  Especificidad  F1-Score      NPV       LR+      LR-  ROC-AUC
Logistic Regression  64   9   3  32  0.888889         0.780488               0.914286       0.876712  0.842105 0.955224  7.415873 0.097768 0.948337
Random Forest (30f)  70   3   5  30  0.925926         0.909091               0.857143       0.958904  0.882353

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

In [13]:
# =============================================================================
# NUEVA SECCIÓN: GRÁFICO COMPARATIVO ROC-AUC
# =============================================================================

print()
print("="*80)
print("SECCIÓN 5B: GRÁFICO COMPARATIVO ROC-AUC")
print("="*80)
print()

# Calcular puntos de la curva para cada modelo
# (Usamos y_test_logit para LR, y_test_trees para los demás)
fpr_lr, tpr_lr, _ = roc_curve(y_test_logit, y_pred_proba_lr)
auc_lr = roc_auc_score(y_test_logit, y_pred_proba_lr)

fpr_rf, tpr_rf, _ = roc_curve(y_test_trees, y_pred_proba_rf)
auc_rf = roc_auc_score(y_test_trees, y_pred_proba_rf)

fpr_xgb, tpr_xgb, _ = roc_curve(y_test_trees, y_pred_proba_xgb)
auc_xgb = roc_auc_score(y_test_trees, y_pred_proba_xgb)

fpr_knn, tpr_knn, _ = roc_curve(y_test_trees, y_pred_proba_knn)
auc_knn = roc_auc_score(y_test_trees, y_pred_proba_knn)

# Iniciar el gráfico
plt.figure(figsize=(18, 12))

# Graficar cada curva
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {auc_lr:.3f})', linewidth=2)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest 30f (AUC = {auc_rf:.3f})', linewidth=2)
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost 30f (AUC = {auc_xgb:.3f})', linewidth=2)
plt.plot(fpr_knn, tpr_knn, label=f'KNN (AUC = {auc_knn:.3f})', linewidth=2, linestyle=':')

# Graficar la línea de no-habilidad (chance)
plt.plot([0, 1], [0, 1], 'k--', label='Azar (AUC = 0.500)')

# Estilo
plt.xlabel('Tasa de Falsos Positivos (1 - Especificidad)', fontsize=12)
plt.ylabel('Tasa de Verdaderos Positivos (Sensibilidad)', fontsize=12)
plt.title('Comparación de Curvas ROC (Test Set)', fontsize=16, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

# Guardar el gráfico
plt.savefig('roc_curves_comparativa1.png', dpi=300, bbox_inches='tight')
plt.close()

print("✓ Guardado: roc_curves_comparativa1.png")
print()


SECCIÓN 5B: GRÁFICO COMPARATIVO ROC-AUC

✓ Guardado: roc_curves_comparativa1.png



In [14]:
# =============================================================================
# NUEVA SECCIÓN: MATRIZ DE CONFUSIÓN DEL MEJOR MODELO
# =============================================================================

print()
print("="*80)
print("SECCIÓN X: MATRIZ DE CONFUSIÓN DEL MEJOR MODELO")
print("="*80)
print()

# Identificar el mejor modelo para la matriz de confusión
# Según tu output, el mejor modelo en validación es Random Forest (30f)
# Necesitamos sus predicciones y la y_test correspondiente

# Obtener los datos del mejor modelo (Random Forest 30f)
# y_pred_best_model = y_pred_rf
# y_test_best_model = y_test_trees
# best_model_name_for_cm = 'Random Forest (30f)'

# O si quieres usar el que tuvo mejor F1 en el test set (XGBoost 41f), aunque en CV no fue el mejor
y_pred_best_model = y_pred_xgb_full
y_test_best_model = y_test_trees
best_model_name_for_cm = 'XGBoost (41f) - Test Set'


cm = confusion_matrix(y_test_best_model, y_pred_best_model)

plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['No SOP', 'SOP'], yticklabels=['No SOP', 'SOP'],
            linewidths=0.5, linecolor='black')
plt.xlabel('Predicción', fontsize=12, fontweight='bold')
plt.ylabel('Real', fontsize=12, fontweight='bold')
plt.title(f'Matriz de Confusión: {best_model_name_for_cm}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('confusion_matrix_best_model.png', dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Guardado: confusion_matrix_best_model.png para {best_model_name_for_cm}")
print()


SECCIÓN X: MATRIZ DE CONFUSIÓN DEL MEJOR MODELO

✓ Guardado: confusion_matrix_best_model.png para XGBoost (41f) - Test Set



In [16]:
# ==========================================================
# SHAP: Fuerza para una predicción individual (XGBoost)
# ==========================================================
print("Generando gráfico SHAP para una predicción individual (XGBoost)...")

# Elige un índice de ejemplo. Por ejemplo, la primera paciente del test set.
# Puedes cambiar este índice (por ejemplo, 5, 10, etc.)
sample_idx = 0 

# Obtener el valor SHAP base (expected_value) y los shap_values para el ejemplo
# Nota: 'explainer_xgb' se creó en la sección anterior de SHAP para XGBoost
expected_value_xgb = explainer_xgb.expected_value

# Asegurarnos de que estamos usando los datos correctos (NumPy array)
# X_test_trees ya es un array de NumPy limpio en este punto del script corregido

plt.figure(figsize=(12, 6))
shap.force_plot(expected_value_xgb, 
                shap_values_xgb[sample_idx,:], 
                X_test_trees[sample_idx,:], 
                feature_names=feature_names_trees, 
                matplotlib=True, show=False)
plt.title(f'SHAP: Predicción Individual (Paciente {sample_idx}, XGBoost)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'shap_xgboost_force_plot_sample_{sample_idx}.png', dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ shap_xgboost_force_plot_sample_{sample_idx}.png")
print()

Generando gráfico SHAP para una predicción individual (XGBoost)...
✓ shap_xgboost_force_plot_sample_0.png



<Figure size 1200x600 with 0 Axes>